## Check Data Correctness

In [6]:
import json
import os
from pymilvus import MilvusClient

# --- CẤU HÌNH ---
# Đường dẫn tính từ folder 'test_eval' đi ra ngoài một cấp (..) rồi vào 'inputs'
INPUT_FILE_PATH = '../inputs/gold_standard_dataset.json' 

# Cấu hình Milvus
MILVUS_URI = "http://127.0.0.1:19530"
COLLECTION_NAME = "globaltech_nlp_project"

print(f"📂 Đang kiểm tra file: {os.path.abspath(INPUT_FILE_PATH)}")

# Kiểm tra file có tồn tại không
if not os.path.exists(INPUT_FILE_PATH):
    print("❌ LỖI: Không tìm thấy file input. Hãy kiểm tra lại đường dẫn.")
else:
    print("✅ Đã tìm thấy file input.")

📂 Đang kiểm tra file: e:\My Document\GlobalTechJSC_NLP\inputs\gold_standard_dataset.json
✅ Đã tìm thấy file input.


In [7]:
def load_and_extract_ids(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        # Xử lý cả trường hợp file là JSON list hoặc JSONL
        try:
            data = json.load(f) # Dạng chuẩn [...]
        except json.JSONDecodeError:
            f.seek(0)
            data = [json.loads(line) for line in f] # Dạng JSONL

    all_ground_truth_ids = set()
    query_map = {} # Map để tra cứu ngược: doc_id này thuộc câu hỏi nào

    for idx, item in enumerate(data):
        q_id = item.get('id', str(idx))
        
        # Lấy danh sách ground_truths
        gts = item.get('ground_truths', [])
        
        for gt in gts:
            # Xử lý linh hoạt: gt có thể là dict hoặc string/int
            doc_id = str(gt['doc_id']) if isinstance(gt, dict) and 'doc_id' in gt else str(gt)
            
            all_ground_truth_ids.add(doc_id)
            
            # Lưu lại để lát nữa báo cáo xem doc_id bị thiếu thuộc câu hỏi nào
            if doc_id not in query_map:
                query_map[doc_id] = []
            query_map[doc_id].append(q_id)
            
    return list(all_ground_truth_ids), query_map, len(data)

# Thực thi
target_ids, query_map, total_queries = load_and_extract_ids(INPUT_FILE_PATH)
print(f"📊 Tổng số câu query: {total_queries}")
print(f"🔍 Tổng số doc_id duy nhất cần kiểm tra (Ground Truths): {len(target_ids)}")

📊 Tổng số câu query: 100
🔍 Tổng số doc_id duy nhất cần kiểm tra (Ground Truths): 357


In [5]:
client = MilvusClient(uri=MILVUS_URI)

def check_existence_in_milvus(doc_ids, collection_name, batch_size=1000):
    found_ids = set()
    total = len(doc_ids)
    
    print(f"🚀 Bắt đầu kiểm tra {total} IDs trong Milvus...")
    
    for i in range(0, total, batch_size):
        batch = doc_ids[i : i + batch_size]
        
        # Query Milvus xem những ID này có tồn tại không
        # Lưu ý: Trường chứa ID bài báo trong Milvus của bạn là 'original_id' hay 'id'?
        # Dựa trên code cũ của bạn là 'original_id'.
        res = client.query(
            collection_name=collection_name,
            filter=f"original_id in {batch}", 
            output_fields=["original_id"]
        )
        
        # Thu thập những ID tìm thấy
        for item in res:
            found_ids.add(str(item['original_id']))
            
    return found_ids

# Thực thi kiểm tra
found_ids = check_existence_in_milvus(target_ids, COLLECTION_NAME)
missing_ids = set(target_ids) - found_ids

print("-" * 30)
print(f"✅ Tìm thấy: {len(found_ids)} bài.")
print(f"❌ Bị thiếu (Missing): {len(missing_ids)} bài.")
print("-" * 30)

🚀 Bắt đầu kiểm tra 357 IDs trong Milvus...
------------------------------
✅ Tìm thấy: 357 bài.
❌ Bị thiếu (Missing): 0 bài.
------------------------------


In [6]:
if len(missing_ids) == 0:
    print("🎉 TUYỆT VỜI! 100% dữ liệu Ground Truth đều tồn tại trong Milvus.")
else:
    print("⚠️ CẢNH BÁO: Các câu hỏi sau đây có đáp án KHÔNG TỒN TẠI trong Database:")
    print("=" * 60)
    
    # Gom nhóm theo câu hỏi
    affected_queries = {}
    for missing in missing_ids:
        query_ids = query_map.get(missing, [])
        for q_id in query_ids:
            if q_id not in affected_queries:
                affected_queries[q_id] = []
            affected_queries[q_id].append(missing)
            
    # In ra danh sách
    count = 0
    for q_id, ids in affected_queries.items():
        count += 1
        print(f"{count}. Query ID: {q_id}")
        print(f"   - Missing Docs: {ids}")
        if count >= 20:
            print("... (và còn nhiều câu khác, chỉ hiển thị 20 câu đầu) ...")
            break

    # Tính tỷ lệ phần trăm dữ liệu hỏng
    missing_ratio = (len(missing_ids) / len(target_ids)) * 100
    print("=" * 60)
    print(f"📉 Tỷ lệ dữ liệu bị thiếu (Data Loss): {missing_ratio:.2f}%")
    print("👉 Khuyến nghị: Bạn cần kiểm tra lại quy trình Insert dữ liệu vào Milvus.")
    print("   Có thể những bài báo này bị lỗi khi crawl hoặc bị lỗi khi embedding nên chưa được nạp vào.")

🎉 TUYỆT VỜI! 100% dữ liệu Ground Truth đều tồn tại trong Milvus.


## Test Qwen: Qwen3 30B A3B Instruct 2507

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

completion = client.chat.completions.create(
    extra_body={},
    model="qwen/qwen3-30b-a3b-instruct-2507",
    messages=[
        {
            "role": "user",
            "content": "Who won the world cup in 2018?"
        }
    ]
)
print(completion.choices[0].message.content)

The winner of the 2018 FIFA World Cup was **France**. They defeated Croatia 4-2 in the final, which was held on July 15, 2018, at the Luzhniki Stadium in Moscow, Russia. This victory marked France's second World Cup title, their first since 1998. Kylian Mbappé and Antoine Griezmann were key players for France in the tournament.


## RAG

In [8]:
import os
import json
import time
import torch
import gc
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import logging as transformers_logging

load_dotenv()

# --- CONFIGURATION ---
transformers_logging.set_verbosity_error()

# Load environment variables
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY") 

# Models Config
EMBED_MODEL_NAME = "hiieu/halong_embedding"
RERANKER_MODEL_NAME = "itdainb/PhoRanker"
LLM_MODEL_NAME = "qwen/qwen3-30b-a3b-instruct-2507" # Model bạn chọn

# Milvus Config
MILVUS_URI = "http://127.0.0.1:19530"
COLLECTION_NAME = "globaltech_nlp_project"

# Paths
INPUT_TEST_FILE = '../inputs/gold_standard_dataset.json' # Đường dẫn file input
OUTPUT_CSV_FILE = '../outputs/rag_final_answers.csv' # Đường dẫn file output

# --- SETUP DEVICES ---
torch.cuda.empty_cache()
gc.collect()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device.upper()}")

# --- LOAD MODELS ---
print("1. Loading Embedding Model...")
embed_model = SentenceTransformer(EMBED_MODEL_NAME, device=device)

print("2. Loading Reranker Model...")
reranker = CrossEncoder(RERANKER_MODEL_NAME, max_length=256, device=device)

print("3. Connecting to Milvus...")
milvus_client = MilvusClient(uri=MILVUS_URI)
milvus_client.load_collection(COLLECTION_NAME)

print("4. Setting up LLM Client...")
llm_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

print("Setup Complete!")

Device: CUDA
1. Loading Embedding Model...
2. Loading Reranker Model...
3. Connecting to Milvus...
4. Setting up LLM Client...
Setup Complete!


In [9]:
def retrieve_and_rerank(query_text, top_k_candidates=20, final_top_k=5):
    """
    Tìm kiếm và sắp xếp lại các tài liệu liên quan nhất.
    Trả về: List các đoạn văn bản (text) tốt nhất.
    """
    # 1. Retrieval (Vector Search)
    query_vector = embed_model.encode([query_text])
    search_res = milvus_client.search(
        collection_name=COLLECTION_NAME,
        data=query_vector,
        limit=top_k_candidates,
        search_params={"metric_type": "COSINE", "params": {"nprobe": 64}},
        output_fields=["title", "text"]
    )
    hits = search_res[0]
    if not hits: return []

    # 2. Re-ranking
    # Tạo cặp [Query, Doc] để chấm điểm
    cross_inp = [[query_text, hit['entity']['text']] for hit in hits]
    cross_scores = reranker.predict(cross_inp)
    
    # Gán điểm và sort
    for idx, hit in enumerate(hits):
        hit['cross_score'] = cross_scores[idx]
    
    sorted_hits = sorted(hits, key=lambda x: x['cross_score'], reverse=True)
    
    # Lấy top K văn bản tốt nhất để làm context
    best_docs = [hit['entity']['text'] for hit in sorted_hits[:final_top_k]]
    return best_docs

def generate_answer(query, context_docs):
    """
    Gọi LLM để sinh câu trả lời dựa trên context.
    """
    if not context_docs:
        context_str = "Không tìm thấy thông tin liên quan."
    else:
        # Nối các đoạn văn bản lại, cách nhau bằng xuống dòng
        context_str = "\n\n".join([f"Document {i+1}: {doc}" for i, doc in enumerate(context_docs)])

    # Prompt Template
    system_prompt = """Bạn là một trợ lý AI hữu ích. Hãy trả lời câu hỏi của người dùng dựa trên thông tin được cung cấp trong phần Context bên dưới.
Nếu thông tin không có trong Context, hãy nói 'Tôi không tìm thấy thông tin này trong dữ liệu được cung cấp'."""
    
    user_prompt = f"""Context:
{context_str}

Question: 
{query}

Answer:"""

    try:
        completion = llm_client.chat.completions.create(
            extra_body={},
            model=LLM_MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating answer: {e}"

# Hàm chạy RAG cho 1 query
def run_rag_pipeline(query):
    # Bước 1: Lấy context (Top 5 docs tốt nhất sau khi rerank)
    context_docs = retrieve_and_rerank(query, top_k_candidates=100, final_top_k=5)
    
    # Bước 2: Sinh câu trả lời
    answer = generate_answer(query, context_docs)
    return answer

In [ ]:
def load_test_data(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            # Xử lý cả format JSON list [...] và JSONL (mỗi dòng 1 json)
            try:
                content = f.read()
                data = json.loads(content)
            except json.JSONDecodeError:
                f.seek(0)
                for line in f:
                    if line.strip():
                        data.append(json.loads(line))
        return data
    except Exception as e:
        print(f"Error loading file: {e}")
        return []

# --- MAIN EXECUTION ---
print("Loading Data...")
test_cases = load_test_data(INPUT_TEST_FILE)


results = []

print(f"Starting RAG Generation for {len(test_cases)} queries...")
print("-" * 50)

for idx, case in enumerate(test_cases):
    query = case.get('question') or case.get('query')
    if not query: continue
    
    print(f"[{idx+1}/{len(test_cases)}] Processing: {query[:50]}...", end="\r")
    
    # Chạy pipeline
    t0 = time.time()
    answer = run_rag_pipeline(query)
    latency = time.time() - t0
    
    results.append({
        "query": query,
        "answer": answer,
        # "latency": round(latency, 2) # Bỏ comment nếu muốn lưu thêm thời gian chạy
    })

print(f"\nCompleted processing {len(results)} queries.")

# --- EXPORT TO CSV ---
df = pd.DataFrame(results)
os.makedirs(os.path.dirname(OUTPUT_CSV_FILE), exist_ok=True)
df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')

print(f"Results saved to: {OUTPUT_CSV_FILE}")
print("\nPreview (First 5 rows):")
print(df.head().to_string())

📥 Loading Data...
🚀 Starting RAG Generation for 100 queries...
--------------------------------------------------
[100/100] Processing: Logo táo trên iPhone 17 Pro Max sẽ được đặt ở đâu?...
✅ Completed processing 100 queries.
💾 Results saved to: ../outputs/rag_final_answers.csv

🔍 Preview (First 5 rows):
                                                                                                                    query                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             